# Adding features - final dataframe
For the most common words there are far more mentions than job posts. Certain keywords are mentioned multiple times in a post. This will skew the results. We need to search each post to see if the keyword is present and return true or false. The assumption being that if a keyword is mentioned in a job post. That is a desired skill.
- Search each post and create dataframe 
- Calculate percentage of posts for each keyword

In [1]:
import numpy as np
import pandas as pd 
import re  
import popular_data_skills.utils.keywords as kws
import popular_data_skills.config.config as config


First lets get the keywords and keyword groups

In [2]:
keywords = kws.Keywords()
keywords.read_file(config.KEYWORDS)

keyword_groups =  kws.Keywords()
keyword_groups.read_file(config.KEYWORD_GROUPS)

Next we will get the dataframe

In [3]:
df = pd.read_csv(config.WRANGLED_DATA_FILE)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   job_title              593 non-null    object 
 1   company_name           591 non-null    object 
 2   location               593 non-null    object 
 3   date_posted            593 non-null    object 
 4   applicant_count        480 non-null    float64
 5   job_description_lines  592 non-null    object 
 6   country                593 non-null    object 
 7   job                    593 non-null    object 
 8   experience             309 non-null    float64
dtypes: float64(2), object(7)
memory usage: 41.8+ KB


In [4]:
# Drop na lines from job description column
df.dropna(subset=['job_description_lines'], inplace=True)

## Search each post and create dataframe 
Now we will create a dataframe identifing if a post has a keyword or not. We will use a regex expression to search for each keyword and their related similiar words in each post. 
- First we need to add spaces either side of each keyword to ensure we search for just that words and not part of another words. r being the best example. 
- Next we will group each keyword and it's related similar words into one expression with an or operator 
- Finally we will search if each job description contains any of these keywords and record the results in a data frame.
- 

In [5]:
# Add spaces to either side of the word
def add_spaces(keyword_dict: dict):
    """Adds one space either side of every string in a dict of lists. 

    Args:
        keyword_dict (dict): _description_

    Returns:
        spaced_kewords (dict): _description_
    """
    spaced_keywords={}
    for key, word_list in keyword_dict.dict.items():
        # Add spaces in keyword 
        spaced_key = key.center(len(key)+2)
        spaced_words_list=[]
        for word in word_list:
            # Add spaces to each similar word
            spaced_word = word.center(len(word)+2)
            spaced_words_list.append(spaced_word )
        spaced_keywords[spaced_key] = spaced_words_list
    return spaced_keywords
        
spaced_keywords = add_spaces(keywords)


In [6]:
# Create df from dict of keywords and similar words 
df_from_dict = pd.DataFrame()

for new_keyword, similar_words in spaced_keywords.items():
    if len(similar_words) > 0:
        pattern = '|'.join(similar_words)
        pattern = pattern +'|'+ new_keyword
    else:
        pattern = new_keyword
    df_from_dict[new_keyword] = df['job_description_lines'].str.contains(pattern, regex=True)
    df_from_dict[new_keyword] = df_from_dict[new_keyword].astype(int)

df_from_dict.head()

,a/b,advertising,architecture,automation,aws,azure,bash,bayesian,c,c#,...,masters degree,bachelor degree,power bi,problem solving,decision trees,random forest,structured data,relational data,unstructured data,unsupervised learning
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [7]:
# Merge from_dict_df and df to get a complete df we can save and use fro further analysis
final_df = pd.merge(df, df_from_dict, left_index=True, right_index=True )
# strip trailing and leading whitespace used in regex patterns
final_df.columns = final_df.columns.str.strip()
final_df 

,job_title,company_name,location,date_posted,applicant_count,job_description_lines,country,job,experience,a/b,...,masters degree,bachelor degree,power bi,problem solving,decision trees,random forest,structured data,relational data,unstructured data,unsupervised learning
0,data engineer / machine learning engineer,streamba,glasgow scotland united kingdom,1 month ago,7.0,\n as a data engineer / machine l...,uk,scientist,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1,data analyst support scientist – remote genomi...,hireresources,santa ana ca,3 days ago,5.0,\none of our valued clients a leading and grow...,usa,scientist,2.0,0,...,1,0,0,1,0,0,0,0,0,0
2,data scientist remote,yelp,glasgow scotland united kingdom,4 days ago,116.0,\n at yelp it’s our mission to co...,uk,scientist,2.0,0,...,0,0,0,0,0,0,0,0,0,0
3,data scientist - growth,spotify,london england united kingdom,2 weeks ago,NaN,\n the freemium r&d team oversees...,uk,scientist,3.0,1,...,0,0,1,0,0,0,0,0,0,0
4,data scientist - growth strategy,spotify,london england united kingdom,2 weeks ago,NaN,\n are you a talented data scient...,uk,scientist,3.0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,analytics developer,dice,united states,28 minutes ago,11.0,\n dice is the leading career des...,usa,analyst,3.0,0,...,0,0,1,0,0,0,0,0,0,0
589,data engineer analystics remote,koch industries,dallas tx,1 week ago,9.0,\ndescription\n\nsentient energy www.sentient-...,usa,analyst,2.0,0,...,0,0,1,0,0,0,0,1,0,0
590,digital analyst - data specialist,ambit consulting services,san francisco ca,2 months ago,10.0,\n acs is a consulting firm focus...,usa,analyst,2.0,0,...,0,0,1,0,0,0,0,1,0,0
591,data analyst ii risk adjustment,boston medical center healthnet plan/well sens...,boston ma,6 hours ago,4.0,\n finance / accounting\n\nremote...,usa,analyst,NaN,0,...,1,0,0,0,0,0,0,0,0,0


## Adding degree column
To make the dataframe easier to filter we will add a degree level column. Stating the level of degree asked if any.   
We will also group the experience into groups rather than individual years

In [8]:
def add_degree(row):
    if row['phd'] == 1:
        return 'phd'
    elif row['masters degree'] == 1:
        return 'ms'
    elif row['bachelor degree'] == 1:
        return 'bs'
    elif row['degree'] == 1:
        return 'any'
    else:
        return 'not specified'

final_df['degree_level'] = final_df.apply(add_degree, axis=1 )
final_df

,job_title,company_name,location,date_posted,applicant_count,job_description_lines,country,job,experience,a/b,...,bachelor degree,power bi,problem solving,decision trees,random forest,structured data,relational data,unstructured data,unsupervised learning,degree_level
0,data engineer / machine learning engineer,streamba,glasgow scotland united kingdom,1 month ago,7.0,\n as a data engineer / machine l...,uk,scientist,NaN,0,...,0,0,0,0,0,0,0,0,0,any
1,data analyst support scientist – remote genomi...,hireresources,santa ana ca,3 days ago,5.0,\none of our valued clients a leading and grow...,usa,scientist,2.0,0,...,0,0,1,0,0,0,0,0,0,phd
2,data scientist remote,yelp,glasgow scotland united kingdom,4 days ago,116.0,\n at yelp it’s our mission to co...,uk,scientist,2.0,0,...,0,0,0,0,0,0,0,0,0,not specified
3,data scientist - growth,spotify,london england united kingdom,2 weeks ago,NaN,\n the freemium r&d team oversees...,uk,scientist,3.0,1,...,0,1,0,0,0,0,0,0,0,any
4,data scientist - growth strategy,spotify,london england united kingdom,2 weeks ago,NaN,\n are you a talented data scient...,uk,scientist,3.0,0,...,0,1,0,1,0,0,0,0,0,not specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,analytics developer,dice,united states,28 minutes ago,11.0,\n dice is the leading career des...,usa,analyst,3.0,0,...,0,1,0,0,0,0,0,0,0,any
589,data engineer analystics remote,koch industries,dallas tx,1 week ago,9.0,\ndescription\n\nsentient energy www.sentient-...,usa,analyst,2.0,0,...,0,1,0,0,0,0,1,0,0,any
590,digital analyst - data specialist,ambit consulting services,san francisco ca,2 months ago,10.0,\n acs is a consulting firm focus...,usa,analyst,2.0,0,...,0,1,0,0,0,0,1,0,0,any
591,data analyst ii risk adjustment,boston medical center healthnet plan/well sens...,boston ma,6 hours ago,4.0,\n finance / accounting\n\nremote...,usa,analyst,NaN,0,...,0,0,0,0,0,0,0,0,0,ms


## Grouping Experience

In [9]:
def group_experience(row):
    if row['experience'] == 0:
        return '0'
    elif row['experience'] == 1:
        return '1'
    elif row['experience'] == 2:
        return '2'
    elif (row['experience'] >= 3) & (row['experience'] < 5):
        return '3-5'
    elif row['experience'] >= 5:
        return '5+'
    else:
        return '0'

final_df['experience'] = final_df.apply(group_experience, axis=1 )
final_df

,job_title,company_name,location,date_posted,applicant_count,job_description_lines,country,job,experience,a/b,...,bachelor degree,power bi,problem solving,decision trees,random forest,structured data,relational data,unstructured data,unsupervised learning,degree_level
0,data engineer / machine learning engineer,streamba,glasgow scotland united kingdom,1 month ago,7.0,\n as a data engineer / machine l...,uk,scientist,0,0,...,0,0,0,0,0,0,0,0,0,any
1,data analyst support scientist – remote genomi...,hireresources,santa ana ca,3 days ago,5.0,\none of our valued clients a leading and grow...,usa,scientist,2,0,...,0,0,1,0,0,0,0,0,0,phd
2,data scientist remote,yelp,glasgow scotland united kingdom,4 days ago,116.0,\n at yelp it’s our mission to co...,uk,scientist,2,0,...,0,0,0,0,0,0,0,0,0,not specified
3,data scientist - growth,spotify,london england united kingdom,2 weeks ago,NaN,\n the freemium r&d team oversees...,uk,scientist,3-5,1,...,0,1,0,0,0,0,0,0,0,any
4,data scientist - growth strategy,spotify,london england united kingdom,2 weeks ago,NaN,\n are you a talented data scient...,uk,scientist,3-5,0,...,0,1,0,1,0,0,0,0,0,not specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,analytics developer,dice,united states,28 minutes ago,11.0,\n dice is the leading career des...,usa,analyst,3-5,0,...,0,1,0,0,0,0,0,0,0,any
589,data engineer analystics remote,koch industries,dallas tx,1 week ago,9.0,\ndescription\n\nsentient energy www.sentient-...,usa,analyst,2,0,...,0,1,0,0,0,0,1,0,0,any
590,digital analyst - data specialist,ambit consulting services,san francisco ca,2 months ago,10.0,\n acs is a consulting firm focus...,usa,analyst,2,0,...,0,1,0,0,0,0,1,0,0,any
591,data analyst ii risk adjustment,boston medical center healthnet plan/well sens...,boston ma,6 hours ago,4.0,\n finance / accounting\n\nremote...,usa,analyst,0,0,...,0,0,0,0,0,0,0,0,0,ms


In [ ]:
final_df.to_csv(config.FINAL_DATA_FILE, index=False)

## Calculate percentage of posts for each keyword

We will now calcualte the percentage of post that contain each keyword. We can show results in groups to better compare keywords. 

In [39]:
skill_counts = df_from_dict.sum()
skill_counts.sort_values(ascending=False, inplace=True)
total_posts = len(df)
pct_skills = round((skill_counts / total_posts) *100, 0)

In [43]:

keyword_groups.dict.keys()

dict_keys(['Topic', 'Skills', 'Business_area', 'Languages', 'Tools', 'Degree'])

In [44]:
in_demand_tools = pct_skills[pct_skills.index.isin(keyword_groups.dict['Tools'])]
in_demand_tools

power bi       29.0
tableau        21.0
excel          16.0
microsoft      13.0
aws            13.0
spark           9.0
azure           8.0
tensorflow      7.0
numpy           6.0
hadoop          6.0
snowflake       6.0
pytorch         5.0
oracle          5.0
scikitlearn     0.0
dtype: float64

In [45]:
in_demand_degree = pct_skills[pct_skills.index.isin(keyword_groups.dict['Degree'])]
in_demand_degree

degree             52.0
masters degree     19.0
phd                10.0
bachelor degree     0.0
dtype: float64

In [46]:
in_demand_languages = pct_skills[pct_skills.index.isin(keyword_groups.dict['Languages'])]
in_demand_languages

python        53.0
sql           51.0
r             30.0
sas            8.0
c              3.0
linux          3.0
javascript     1.0
bash           1.0
c#             1.0
perl           1.0
dtype: float64

In [47]:
in_demand_business = pct_skills[pct_skills.index.isin(keyword_groups.dict['Business_area'])]
in_demand_business

healthcare     53.0
finance        31.0
insurance      21.0
marketing      18.0
sales          12.0
security       10.0
law             8.0
government      8.0
advertising     4.0
crm             3.0
economic        2.0
dtype: float64

In [48]:
in_demand_skills = pct_skills[pct_skills.index.isin(keyword_groups.dict['Skills'])]
in_demand_skills 

visualization            45.0
deep                     23.0
decision trees           19.0
automation               18.0
cloud                    17.0
optimization             13.0
regression               11.0
mining                   11.0
relational data          10.0
unstructured data        10.0
structured data           9.0
pandas                    9.0
classification            8.0
a/b                       8.0
deploy                    8.0
nlp                       8.0
recommendation            7.0
clustering                7.0
neural                    6.0
architecture              6.0
pipeline                  5.0
supervised                5.0
unsupervised learning     5.0
random forest             4.0
bayesian                  4.0
dashboard                 3.0
dtype: float64

In [49]:
in_demand_subjects = pct_skills[pct_skills.index.isin(keyword_groups.dict['Topic'])]
in_demand_subjects

analytics          90.0
models             60.0
statistics         59.0
ML                 52.0
coding             40.0
databases          28.0
training           20.0
algorithms         20.0
testing            20.0
problem solving    19.0
AI                 17.0
documentation      10.0
forecasting         7.0
preparation         4.0
predictions         2.0
dtype: float64

There we have the percentage of posts that ask for each keyword. Next we will make this data frame filterable to use in a dashboard to better visualise our findings. 

There are still some refinements we could make in the future. power bi and microsoft suite as well as google suite may need looking into. We could possible group than together into suites.
search terms might be throwing false positives and may need some looking into. bi, c, decision may need another word in the search term to narrow down the keyword. 
terms may be able to be grouped together like relational and structued data. 